
## 텐서플로 세션의 구조

텐서플로 라이브러리의 내용을 설명하기 앞서, 파이썬이나 C/C++와 같은 언어를 다루다가 텐서플로를 사용하기 시작하려면 가장 먼저 이해해야 하는것은 바로 그래프와 세션입니다.

### 그래프

텐서플로는 기본적으로 모든 데이터의 흐름을 그래프로 나타내는것으로 머신러닝 과정을 나타냅니다. 각 상수, 변수, 행렬, 텐서(고차원행렬)는 하나의 노드라고 생각해야하고, 계산식이나 함수들은 노드 사이를 연결해주는 가지라고 생각해야합니다. 즉 텐서플로로 구축된 모델은 하나의 시작점에서 시작한 방향성 그래프라고 보면 됩니다.

### 세션

세션은 그래프나 그래프의 일부분을 '실행'하는 공간입니다. 텐서플로가 이렇게 계산형태와 계산과정을 분리한 이유는 머신러닝의 특성과 밀접하게 관련이 있습니다. 머신러닝은 기본적으로 '모델링'과 '트레이닝,' 그리고 '예측'의 과정으로 이루어져있습니다. 그렇다보니 '계산의 결과'보다 '계산의 형태'를 저장하고 수정하는게 더 효과적일 수 있습니다.
